In [1]:
# Find top 50 market capitalization in SPY

import pandas as pd
import os
import numpy as np
import datetime as dt
import pickle

import pandas as pd
URL = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
tickers = pd.read_html(URL)[0]['Symbol'].tolist()
tickers

ticker_list = [
    'AAPL',
    'MSFT',
    'AMZN',
    'GOOGL',
    'NVDA',
    'TSLA',
    'META',
    'GOOG',
    'XOM',
    'UNH',
    'JPM',
    'LLY',
    'JNJ',
    'V',
    'PG',
    'AVGO',
    'MA',
    'HD',
    'CVX',
    'ABBV',
    'MRK',
    'COST',
    'PEP',
    'WMT',
    'ADBE',
    'CSCO',
    'KO',
    'CRM',
    'TMO',
    'ACN'
]


In [2]:
#download ticker

In [3]:
import pandas_ta as ta
import yfinance as yf

for ticker in ticker_list:
    print(f"Processing {ticker}")
    df = yf.download(ticker, start="2015-01-01", end="2021-01-01")

    #df["RSI(2)"] = ta.rsi(df['Close'], length=2)
    df["RSI(7)"] = ta.rsi(df['Close'], length=7)
    #df["RSI(14)"] = ta.rsi(df['Close'], length=14)
    #df["CCI(30)"] = ta.cci(close=df['Close'],
                       #length=30, high=df["High"], low=df["Low"])
    #df["CCI(50)"] = ta.cci(close=df['Close'],
                       #length=50, high=df["High"], low=df["Low"])
    #df["CCI(100)"] = ta.cci(close=df['Close'],
                        #length=100, high=df["High"], low=df["Low"])
    
    #Generate signals if RSI(2) <= 30, position = 1, if RSI(2) >= 70, position = -1, else position = 0
    df["position"] = np.where(df["RSI(7)"] <= 30, 1, np.where(df["RSI(7)"] >= 70, -1, 0))
    
    #Shift position by 1 day to avoid look ahead bias
    df.position = df.position.shift(1)
    df.dropna(inplace=True)
    print(df.head())    
    df.to_csv(f"data/{ticker}.csv")
    
    
    
    


Processing AAPL
[*********************100%%**********************]  1 of 1 completed
                 Open       High        Low      Close  Adj Close      Volume  \
Date                                                                            
2011-01-12  12.258929  12.301071  12.214286  12.300714  10.440802   302590400   
2011-01-13  12.327143  12.380000  12.280357  12.345714  10.478997   296780400   
2011-01-14  12.353214  12.445714  12.301429  12.445714  10.563878   308840000   
2011-01-18  11.768571  12.312857  11.642857  12.166071  10.326519  1880998000   
2011-01-19  12.441071  12.450000  12.031429  12.101429  10.271648  1135612800   

               RSI(7)  position  
Date                             
2011-01-12  92.827682       0.0  
2011-01-13  93.627538      -1.0  
2011-01-14  95.056758      -1.0  
2011-01-18  54.891734      -1.0  
2011-01-19  49.276567       0.0  
Processing MSFT
[*********************100%%**********************]  1 of 1 completed
                 Open   

In [4]:
#Count number of signals for each ticker
for ticker in ticker_list:
    df = pd.read_csv(f"data/{ticker}.csv")
    sum_signal_buy = df[df['position'] == 1].shape[0]
    sum_signal_sell = df[df['position'] == -1].shape[0]
    
    print(f"{ticker} has {sum_signal_buy} buy signals and {sum_signal_sell} sell signals")

AAPL has 216 buy signals and 649 sell signals
MSFT has 164 buy signals and 452 sell signals
AMZN has 170 buy signals and 528 sell signals
GOOGL has 192 buy signals and 528 sell signals
NVDA has 197 buy signals and 503 sell signals
TSLA has 250 buy signals and 508 sell signals
META has 162 buy signals and 400 sell signals
GOOG has 190 buy signals and 528 sell signals
XOM has 326 buy signals and 288 sell signals
UNH has 157 buy signals and 474 sell signals
JPM has 224 buy signals and 439 sell signals
LLY has 184 buy signals and 475 sell signals
JNJ has 212 buy signals and 497 sell signals
V has 89 buy signals and 468 sell signals
PG has 253 buy signals and 400 sell signals
AVGO has 180 buy signals and 426 sell signals
MA has 109 buy signals and 492 sell signals
HD has 197 buy signals and 558 sell signals
CVX has 315 buy signals and 358 sell signals
ABBV has 179 buy signals and 350 sell signals
MRK has 230 buy signals and 386 sell signals
COST has 176 buy signals and 527 sell signals
PEP 

In [5]:
#Dictionary comprehension to store number of buy and sell signals for each ticker
signal_dict = {ticker: pd.read_csv(f"data/{ticker}.csv",
                                   index_col="Date", parse_dates=True) for ticker in ticker_list}
print(signal_dict)

{'AAPL':                   Open        High         Low       Close   Adj Close  \
Date                                                                     
2011-01-12   12.258929   12.301071   12.214286   12.300714   10.440802   
2011-01-13   12.327143   12.380000   12.280357   12.345714   10.478997   
2011-01-14   12.353214   12.445714   12.301429   12.445714   10.563878   
2011-01-18   11.768571   12.312857   11.642857   12.166071   10.326519   
2011-01-19   12.441071   12.450000   12.031429   12.101429   10.271648   
...                ...         ...         ...         ...         ...   
2020-12-24  131.320007  133.460007  131.100006  131.970001  129.850616   
2020-12-28  133.990005  137.339996  133.509995  136.690002  134.494766   
2020-12-29  138.050003  138.789993  134.339996  134.869995  132.704025   
2020-12-30  135.580002  135.990005  133.399994  133.720001  131.572464   
2020-12-31  134.080002  134.740005  131.720001  132.690002  130.559006   

                Volume     R

In [11]:
import datetime

signal_dict['AAPL'].loc['2011-01-12']
datetime.datetime.strptime(
    '2011-01-12', '%Y-%m-%d') < datetime.datetime.strptime('2020-01-12', '%Y-%m-%d')


True